In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

Using TensorFlow backend.


In [2]:
tf.__version__

'2.2.0'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
training_set = train_datagen.flow_from_directory('dataset/train',
                                                target_size=(64,64),
                                                batch_size=32,
                                                class_mode='binary')

Found 5216 images belonging to 2 classes.


In [4]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_set = validation_datagen.flow_from_directory('dataset/val',
                                                       target_size=(64,64),
                                                       batch_size=32,
                                                       class_mode='binary')

Found 16 images belonging to 2 classes.


In [21]:
cnn = tf.keras.models.Sequential()

In [22]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',
                              input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [23]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                              kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [24]:
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [25]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
cnn.fit(x=training_set,validation_data=validation_set,epochs=25)

Epoch 1/25
163/163 [==============================] - 94s 579ms/step - loss: 0.3205 - accuracy: 0.8643 - val_loss: 0.5552 - val_accuracy: 0.7500
Epoch 2/25
163/163 [==============================] - 95s 581ms/step - loss: 0.2397 - accuracy: 0.9009 - val_loss: 0.4938 - val_accuracy: 0.7500
Epoch 3/25
163/163 [==============================] - 96s 587ms/step - loss: 0.2095 - accuracy: 0.9160 - val_loss: 0.3147 - val_accuracy: 0.8750
Epoch 4/25
163/163 [==============================] - 94s 576ms/step - loss: 0.1826 - accuracy: 0.9304 - val_loss: 0.6221 - val_accuracy: 0.6250
Epoch 5/25
163/163 [==============================] - 94s 579ms/step - loss: 0.1702 - accuracy: 0.9335 - val_loss: 0.3198 - val_accuracy: 0.8125
Epoch 6/25
163/163 [==============================] - 95s 581ms/step - loss: 0.1794 - accuracy: 0.9287 - val_loss: 0.7845 - val_accuracy: 0.6875
Epoch 7/25
163/163 [==============================] - 94s 579ms/step - loss: 0.1536 - accuracy: 0.9413 - val_loss: 0.3938 - val_ac

In [28]:
training_set.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [40]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/test/PNEUMONIA/person11_virus_38.jpeg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
if result[0][0] ==1:
    prediction = 'PNEUMONIA'
else:
    prediction = 'NORMAL'
prediction

'PNEUMONIA'

<h3>Over-Fitting</h3>
<p>
    <ol>
        <li><a href="https://machinelearningmastery.com/introduction-to-regularization-to-reduce-overfitting-and-improve-generalization-error/">General Idea Overfitting</a></li>
        <li><a href="https://androidkt.com/k-fold-cross-validation-with-tensorflow-keras/">How to work Overfitting.</a></li>
    </ol>
</p>

In [6]:
cnn2 = tf.keras.models.Sequential()
cnn2.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',
                              input_shape=[64,64,3]))
cnn2.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
cnn2.add(tf.keras.layers.Conv2D(filters=32,
                              kernel_size=3,activation='relu'))
cnn2.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
cnn2.add(tf.keras.layers.Flatten())
cnn2.add(tf.keras.layers.Dense(units=128,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
cnn2.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
cnn2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
cnn2.fit(x=training_set,validation_data=validation_set,epochs=25)

Epoch 1/25
163/163 [==============================] - 117s 720ms/step - loss: 1.8951 - accuracy: 0.7822 - val_loss: 0.5782 - val_accuracy: 0.8750
Epoch 2/25
163/163 [==============================] - 103s 630ms/step - loss: 0.4182 - accuracy: 0.8556 - val_loss: 0.5778 - val_accuracy: 0.7500
Epoch 3/25
163/163 [==============================] - 100s 613ms/step - loss: 0.3717 - accuracy: 0.8717 - val_loss: 1.2569 - val_accuracy: 0.5625
Epoch 4/25
163/163 [==============================] - 98s 600ms/step - loss: 0.3867 - accuracy: 0.8712 - val_loss: 0.9935 - val_accuracy: 0.6250
Epoch 5/25
163/163 [==============================] - 97s 598ms/step - loss: 0.3439 - accuracy: 0.8840 - val_loss: 0.7160 - val_accuracy: 0.8125
Epoch 6/25
163/163 [==============================] - 99s 609ms/step - loss: 0.3180 - accuracy: 0.8957 - val_loss: 0.5930 - val_accuracy: 0.8125
Epoch 7/25
163/163 [==============================] - 96s 591ms/step - loss: 0.3060 - accuracy: 0.8974 - val_loss: 0.8003 - val

In [11]:
cnn3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
    tf.keras.layers.Dense(units=128,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
    tf.keras.layers.Dense(units=1,activation='sigmoid')
])

cnn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
cnn3.fit(x=training_set,validation_data=validation_set,epochs=25)

Epoch 1/25
163/163 [==============================] - 99s 608ms/step - loss: 5.4394 - accuracy: 0.7699 - val_loss: 0.8931 - val_accuracy: 0.6875
Epoch 2/25
163/163 [==============================] - 97s 597ms/step - loss: 0.4613 - accuracy: 0.8604 - val_loss: 0.5353 - val_accuracy: 0.8750
Epoch 3/25
163/163 [==============================] - 97s 595ms/step - loss: 0.3799 - accuracy: 0.8710 - val_loss: 0.6100 - val_accuracy: 0.8125
Epoch 4/25
163/163 [==============================] - 97s 595ms/step - loss: 0.3568 - accuracy: 0.8825 - val_loss: 0.6858 - val_accuracy: 0.6875
Epoch 5/25
163/163 [==============================] - 100s 613ms/step - loss: 0.3387 - accuracy: 0.8877 - val_loss: 0.5249 - val_accuracy: 0.8750
Epoch 6/25
163/163 [==============================] - 96s 591ms/step - loss: 0.3151 - accuracy: 0.9007 - val_loss: 0.7559 - val_accuracy: 0.6250
Epoch 7/25
163/163 [==============================] - 96s 590ms/step - loss: 0.2875 - accuracy: 0.9095 - val_loss: 0.8541 - val_a

<h3>Finding Correct Hyper-parameters</h3>
<p>
    - Grid Search<br>
    - Cross Validation
</p>

<h3>Training Correct Neural Networks without Over-Fitting</h3>
<p>
    <b>Regression Loss/Cost/Error</b>
       <ol>
           <li>Sum Of Squared Error</li>
        </ol>
        <b>Classification Loss/Cost/Error</b>
       <ol>
           <li>Cross Entropy</li>
        </ol>
         <b>BackPropagation</b>
       <ol>
           <li>Using Chain Rule of gradient descent algorithm</li>
        </ol>
        <b>Regularization</b>
       <ol>
           <li>Dropout regularization</li>
            <li>Early Stopping</li>
        </ol>
        
</p>